# Download and install the ADK and Agent Engine SDK

In [ ]:
!pip3 install google-cloud-aiplatform[agent_engines,adk]==1.88 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.2 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 44.0.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; pl

# Initialization

In [ ]:
import vertexai
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

PROJECT_ID = "reasoning-engine-test-1" #@param {type:"string"}
STAGING_BUCKET = "gs://reasoning-engine-test-1-bucket" #@param {type:"string"}
LOCATION = "us-central1" #@param {type:"string"}

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

# Defining functions
roll_die - rolls dice to create random number

check_prime - checks if a number is prime

In [ ]:
def roll_die(sides: int) -> int:
  """Roll a die and return the rolled result.

  Args:
    sides: The integer number of sides the die has.

  Returns:
    An integer of the result of rolling the die.
  """
  import random

  return random.randint(1, sides)

In [ ]:
def check_prime(nums: list[int]) -> list[str]:
  """Check if a given list of numbers are prime.

  Args:
    nums: The list of numbers to check.

  Returns:
    A str indicating which number is prime.
  """

  primes = set()
  for number in nums:
    number = int(number)
    if number <= 1:
      continue
    is_prime = True
    for i in range(2, int(number**0.5) + 1):
      if number % i == 0:
        is_prime = False
        break
    if is_prime:
      primes.add(number)
  return (
      'No prime numbers found.'
      if not primes
      else f"{', '.join(str(num) for num in primes)} are prime numbers."
  )

# Define an agent in the agent framework

An AI agent reasons, plans, and takes actions. The agent takes actions via access to **tools**, deciding how and when to invoke a tool. The agent also manages orchestration, creating a plan for answering a user query and adapting to responses that aren't quite correct.

Create a simple agent. An agent requires two things: a display name and a set of instructions. The instructions should be detailed, making it clear exactly how the agent should behave.



In [ ]:
from google.adk.agents import Agent

agent = Agent(
    model="gemini-1.5-flash-002",
    name='data_processing_agent',
    instruction="""
      You roll dice and answer questions about the outcome of the dice rolls.
      You can roll dice of different sizes.
      You can use multiple tools in parallel by calling functions in parallel(in one request and in one round).
      The only things you do are roll dice for the user and discuss the outcomes.
      It is ok to discuss previous dice roles, and comment on the dice rolls.
      When you are asked to roll a die, you must call the roll_die tool with the number of sides. Be sure to pass in an integer. Do not pass in a string.
      You should never roll a die on your own.
      When checking prime numbers, call the check_prime tool with a list of integers. Be sure to pass in a list of integers. You should never pass in a string.
      You should not check prime numbers before calling the tool.
      When you are asked to roll a die and check prime numbers, you should always make the following two function calls:
      1. You should first call the roll_die tool to get a roll. Wait for the function response before calling the check_prime tool.
      2. After you get the function response from roll_die tool, you should call the check_prime tool with the roll_die result.
        2.1 If user asks you to check primes based on previous rolls, make sure you include the previous rolls in the list.
      3. When you respond, you must include the roll_die result from step 1.
      You should always perform the previous 3 steps when asking for a roll and checking prime numbers.
      You should not rely on the previous history on prime results.
    """,
    tools=[roll_die, check_prime],
)

app = AdkApp(
    agent=agent,
    enable_tracing=True,
)

app

# Let's try it (locally)

Under the hood, `app: AdkApp` will be using an in-memory session store when running locally.

**Remark**: the operations below (i.e., `.create_session`, `.list_sessions`, `.get_session`, `.stream_query`) are only supported by the `reasoning_engines.AdkApp` class, not the `google.adk.agents.Agent` class.

In [ ]:
session = app.create_session(user_id="ysian")
session

Session(id='999dd8fc-6267-4303-b3c2-053e08e21a08', app_name='default-app-name', user_id='ysian', state={}, events=[], last_update_time=1744165185.8415737)

In [ ]:
app.list_sessions(user_id="ysian")

ListSessionsResponse(sessions=[Session(id='999dd8fc-6267-4303-b3c2-053e08e21a08', app_name='default-app-name', user_id='ysian', state={}, events=[], last_update_time=1744165185.8415737)])

In [ ]:
session = app.get_session(user_id="ysian", session_id=session.id)
session

Session(id='999dd8fc-6267-4303-b3c2-053e08e21a08', app_name='default-app-name', user_id='ysian', state={}, events=[], last_update_time=1744165185.8415737)

In [ ]:
for event in app.stream_query(
    user_id="ysian",
    session_id=session.id,
    message="roll a 2 sided die",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-af72fb97-b7f1-49ad-b1d1-ff7080057845', 'args': {'sides': 2}, 'name': 'roll_die'}}], 'role': 'model'}, 'invocation_id': 'e-9e9eff34-57a0-4499-95f2-7bb7d4b5f6d7', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'Z6gGVyXx', 'timestamp': 1744165186.27417}
{'content': {'parts': [{'function_response': {'id': 'adk-af72fb97-b7f1-49ad-b1d1-ff7080057845', 'name': 'roll_die', 'response': {'result': 2}}}], 'role': 'user'}, 'invocation_id': 'e-9e9eff34-57a0-4499-95f2-7bb7d4b5f6d7', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'baRbhg5m', 'timestamp': 1744165187.380104}
{'content': {'parts': [{'text': 'I rolled a 2 sided die and got a 2.\n'}], 'role': 'model'}, 'invocation_id': 'e-9e9eff34-57a0-4499-95f2-7bb7d4b5f6d7', 'author': 'data_processing_agent', 'actions

You can navigate to https://pantheon.corp.google.com/traces/explorer;project=reasoning-engine-test-1 to view the traces for it (they might take some time to show up).
* If you don't have the permissions to view them in Cloud Trace, please follow the instructions in https://cloud.google.com/trace/docs/finding-traces#before_you_begin to grant yourself permission to use Trace Explorer with the project.

# Deploy to Vertex AI Agent Engine
To call and manage ADK applications in production, deploy the `app` to Vertex AI Agent Engine.

## AdkApp deployment

In [ ]:
remote_app1 = agent_engines.create(
    app,
    requirements=["google-cloud-aiplatform[agent_engines,adk]>=1.88"],
)

INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.88.0'}
INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]>=1.88', 'cloudpickle==3.1.1']
INFO:vertexai.agent_engines:Using bucket reasoning-engine-test-1-bucket
INFO:vertexai.agent_engines:Wrote to gs://reasoning-engine-test-1-bucket/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://reasoning-engine-test-1-bucket/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://reasoning-engine-test-1-bucket/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/349765473021/locations/us-central1/reasoningEngines/9036842088225

**Remark**: You can ignore the following warnings:
* (i) `WARNING:vertexai.agent_engines:Failed to parse constraint: ` and
* (ii) `WARNING:vertexai.agent_engines:The following requirements are missing: {'cloudpickle'}`

We have auto-heal mechanisms to still reflect them in the final list of requirements: `INFO:vertexai.agent_engines:The final list of requirements`

**Remark**: The automatic provisioning of permissions for Vertex AI Sessions have not rolled out yet, so you'll have to follow https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/set-up#service-agent in the meantime to grant "Vertex AI User" (`roles/aiplatform.user`) to the service account in your GCP project

## adk.agents.Agent deployment

Alternatively, you can deploy the `agent` directly without wrapping it into an `AdkApp`. The agent engine SDK will do the necessary transformations for you under-the-hood.

In [ ]:
# Alternatively
remote_app2 = agent_engines.create(
    agent,
    requirements=["google-cloud-aiplatform[agent_engines,adk]>=1.88"],
)

INFO:vertexai.agent_engines:Deploying google.adk.agents.Agent as an application.
INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.88.0'}
INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,adk]>=1.88', 'cloudpickle==3.1.1']
INFO:vertexai.agent_engines:Using bucket reasoning-engine-test-1-bucket
INFO:vertexai.agent_engines:Wrote to gs://reasoning-engine-test-1-bucket/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://reasoning-engine-test-1-bucket/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://reasoning-engine-test-1-bucket/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backin

**Remark**: You can ignore the following warnings:
* (i) `WARNING:vertexai.agent_engines:Failed to parse constraint: ` and
* (ii) `WARNING:vertexai.agent_engines:The following requirements are missing: {'cloudpickle'}`

We have auto-heal mechanisms to still reflect them in the final list of requirements: `INFO:vertexai.agent_engines:The final list of requirements`

**Remark**: The automatic provisioning of permissions for Vertex AI Sessions have not rolled out yet, so you'll have to follow https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/set-up#service-agent in the meantime to grant "Vertex AI User" (`roles/aiplatform.user`) to the service account in your GCP project

# Let's try it (remotely)

## Remote AdkApp

In [ ]:
import pprint

pprint.pprint(remote_app1.operation_schemas())

[{'api_mode': '',
  'description': 'Get a session for the given user.\n'
                 '\n'
                 '        Args:\n'
                 '            user_id (str):\n'
                 '                Required. The ID of the user.\n'
                 '            session_id (str):\n'
                 '                Required. The ID of the session.\n'
                 '            **kwargs (dict[str, Any]):\n'
                 '                Optional. Additional keyword arguments to '
                 'pass to the\n'
                 '                session service.\n'
                 '\n'
                 '        Returns:\n'
                 '            Session: The session instance (if any). It '
                 'returns None if the\n'
                 '            session is not found.\n'
                 '\n'
                 '        Raises:\n'
                 '            RuntimeError: If the session is not found.\n'
                 '        ',
  'name': 'get

In [ ]:
session1 = remote_app1.create_session(user_id="ysian")
session1

{'events': [],
 'user_id': 'ysian',
 'state': {},
 'id': '707008516648337408',
 'app_name': '903684208822583296',
 'last_update_time': 1744165437.450694}

**Remark**: The automatic provisioning of permissions for Vertex AI Sessions have not rolled out yet, so you'll have to follow https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/set-up#service-agent in the meantime to grant "Vertex AI User" (`roles/aiplatform.user`) to the service account in your GCP project.

In [ ]:
remote_app1.list_sessions(user_id="ysian")

{'sessions': [{'events': [],
   'last_update_time': 1744165437.450694,
   'state': {},
   'id': '707008516648337408',
   'app_name': '903684208822583296',
   'user_id': 'ysian'}]}

In [ ]:
session = remote_app1.get_session(user_id="ysian", session_id=session1["id"])
session

{'events': [],
 'last_update_time': 1744165437.450694,
 'state': {},
 'id': '707008516648337408',
 'app_name': '903684208822583296',
 'user_id': 'ysian'}

In [ ]:
for event in remote_app1.stream_query(
    user_id="ysian",
    session_id=session1["id"],
    message="roll a 2 sided die",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-aef83b34-1016-42e2-b68e-373c690e2a79', 'args': {'sides': 2}, 'name': 'roll_die'}}], 'role': 'model'}, 'invocation_id': 'e-23f3a630-22fd-461a-97c6-204886ae4ead', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'IhjRpzxv', 'timestamp': 1744165440.122969}
{'content': {'parts': [{'function_response': {'id': 'adk-aef83b34-1016-42e2-b68e-373c690e2a79', 'name': 'roll_die', 'response': {'result': 1}}}], 'role': 'user'}, 'invocation_id': 'e-23f3a630-22fd-461a-97c6-204886ae4ead', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'gDwrTwzk', 'timestamp': 1744165441.509328}
{'content': {'parts': [{'text': 'I rolled a 2 sided die and got 1.\n'}], 'role': 'model'}, 'invocation_id': 'e-23f3a630-22fd-461a-97c6-204886ae4ead', 'author': 'data_processing_agent', 'actions': {

In [ ]:
for event in remote_app1.stream_query(
    user_id="ysian",
    session_id=session1["id"],
    message="What were the previous rolls?",
):
    print(event)

{'content': {'parts': [{'text': 'The only roll I have performed so far was a roll of a 2 sided die, which resulted in a 1.\n'}], 'role': 'model'}, 'invocation_id': 'e-1d21a6ba-095d-4b72-babd-c1bce882f170', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'gCsL3FGx', 'timestamp': 1744165442.466805}


## Remote adk.agents.Agent

In [ ]:
import pprint

pprint.pprint(remote_app2.operation_schemas())

[{'api_mode': '',
  'description': 'Get a session for the given user.\n'
                 '\n'
                 '        Args:\n'
                 '            user_id (str):\n'
                 '                Required. The ID of the user.\n'
                 '            session_id (str):\n'
                 '                Required. The ID of the session.\n'
                 '            **kwargs (dict[str, Any]):\n'
                 '                Optional. Additional keyword arguments to '
                 'pass to the\n'
                 '                session service.\n'
                 '\n'
                 '        Returns:\n'
                 '            Session: The session instance (if any). It '
                 'returns None if the\n'
                 '            session is not found.\n'
                 '\n'
                 '        Raises:\n'
                 '            RuntimeError: If the session is not found.\n'
                 '        ',
  'name': 'get

In [ ]:
session2 = remote_app2.create_session(user_id="ysian")
session2

{'events': [],
 'user_id': 'ysian',
 'state': {},
 'id': '3125441516546293760',
 'app_name': '3511268393070100480',
 'last_update_time': 1744165808.203766}

**Remark**: The automatic provisioning of permissions for Vertex AI Sessions have not rolled out yet, so you'll have to follow https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/set-up#service-agent in the meantime to grant "Vertex AI User" (`roles/aiplatform.user`) to the service account in your GCP project.

In [ ]:
remote_app2.list_sessions(user_id="ysian")

{'sessions': [{'events': [],
   'last_update_time': 1744165808.203766,
   'state': {},
   'id': '3125441516546293760',
   'app_name': '3511268393070100480',
   'user_id': 'ysian'}]}

In [ ]:
session2 = remote_app2.get_session(user_id="ysian", session_id=session2["id"])
session2

{'events': [],
 'last_update_time': 1744165808.203766,
 'state': {},
 'id': '3125441516546293760',
 'app_name': '3511268393070100480',
 'user_id': 'ysian'}

In [ ]:
for event in remote_app2.stream_query(
    user_id="ysian",
    session_id=session2["id"],
    message="roll a 2 sided die",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-6b6dbbe7-872b-4fcd-a954-f0d474ad2177', 'args': {'sides': 2}, 'name': 'roll_die'}}], 'role': 'model'}, 'invocation_id': 'e-fda28f8a-a752-4589-938d-d73a9850ca8b', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': '3XjRnonv', 'timestamp': 1744165810.466326}
{'content': {'parts': [{'function_response': {'id': 'adk-6b6dbbe7-872b-4fcd-a954-f0d474ad2177', 'name': 'roll_die', 'response': {'result': 2}}}], 'role': 'user'}, 'invocation_id': 'e-fda28f8a-a752-4589-938d-d73a9850ca8b', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'P7sqgatO', 'timestamp': 1744165811.066498}
{'content': {'parts': [{'text': 'I rolled a 2-sided die and got a 2.\n'}], 'role': 'model'}, 'invocation_id': 'e-fda28f8a-a752-4589-938d-d73a9850ca8b', 'author': 'data_processing_agent', 'actions':

In [ ]:
for event in remote_app2.stream_query(
    user_id="ysian",
    session_id=session2["id"],
    message="What were the previous rolls?",
):
    print(event)

{'content': {'parts': [{'text': 'The previous roll was a 2 on a 2-sided die.\n'}], 'role': 'model'}, 'invocation_id': 'e-a6ea3500-1e77-4b82-8500-71b109be8f37', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'OirF6iA6', 'timestamp': 1744165812.068509}


# Clean up

In [ ]:
remote_app1.delete(force=True)

INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/349765473021/locations/us-central1/operations/6795315114758111232
INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/349765473021/locations/us-central1/reasoningEngines/903684208822583296


In [ ]:
remote_app2.delete(force=True)

INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/349765473021/locations/us-central1/operations/577532879219720192
INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/349765473021/locations/us-central1/reasoningEngines/3511268393070100480
